In [1]:
import spacy
import nltk
import pandas as pd

In [2]:
from collections import Counter

In [3]:
data = pd.read_csv(r"D:\Data\wikipedia-ml\wikipedia_machine_learning.csv",sep='\t')

The format of the data is that each row is a series, and each series has 3 elements: 0:title, 1: url, 2: body text 

In [4]:
d0 = data.loc[0]

In [5]:
type(d0)

pandas.core.series.Series

In [6]:
d0[0]

'Outline of computer vision'

In [7]:
d0[1]

'https://en.wikipedia.org/wiki/Outline_of_computer_vision'

In [8]:
d0[2][:100]

'The following outline is provided as an overview of and topical guide to computer vision: Computer v'

In [9]:
data.shape

(7318, 3)

In [10]:
articles = data.apply(lambda x: x[2], axis=1)

In [11]:
titles = data.apply(lambda x: x[0], axis=1)

In [12]:
urls = data.apply(lambda x: x[1], axis=1)

In [13]:
df = pd.concat([titles,urls,articles],axis=1)

In [14]:
df.columns = ['title', 'url','original_article']

The next stage is to apply typical NLP preprocessing steps before looking at what words tend to be used often:
- lowercase
- remove stopwords

In [16]:
list(set([type(a) for a in df.original_article]))

[float, str]

In [17]:
float_cols = df[df['original_article'].apply(lambda x: isinstance(x, float))]

In [18]:
float_cols.head()

,title,url,original_article
100,Category:Robotics suites,https://en.wikipedia.org/wiki/Category:Robotic...,NaN
5449,Category:Search algorithms,https://en.wikipedia.org/wiki/Category:Search_...,NaN


In [20]:
df['original_article'].isna().sum()

2

There's not many nulls, I'm just going to drop them, I'd check the proportion of nulls, but since the number of instances is in the thousands, and this is 2, I'm just going to drop them

In [21]:
df = df.dropna(how='any')
df.shape

(7316, 3)

In [23]:
articles = df['original_article'].apply(lambda s: s.lower())

I need to create a dataframe with a column for each word, where the row is the count of that word in that article

In [24]:
 from nltk.corpus import stopwords

In [25]:
articles = articles.apply(lambda row: nltk.word_tokenize(row))

In [26]:
articles.head()

0    [the, following, outline, is, provided, as, an...
1    [the, following, outline, is, provided, as, an...
2    [the, following, outline, is, provided, as, an...
3    [the, accuracy, paradox, is, the, paradoxical,...
4    [action, model, learning, (, sometimes, abbrev...
Name: original_article, dtype: object

In [27]:
stop_words = set(stopwords.words('english')) 

I was conservative with removing the punctuation, because some of it might be important, like ? especially, and ==

In [42]:
punctuation = "()'',.:"

I need to remove punctuation as well as stopwords

In [ ]:
def remove_unwanted(tokens):
    tokens = [w for w in t if w not in stop_words and w not in punctuation]
    return tokens

In [43]:
punctuation

"()'',.:"

In [52]:
a = articles.loc[0]

In [44]:
articles = articles.apply(lambda x: [w for w in x if w not in stop_words and w not in punctuation])

In [45]:
print(articles.shape)
print(df.shape)

(7316,)
(7316, 5)


In [53]:
c = Counter(a)

In [55]:
df['orig_']

['following',
 'outline',
 'provided',
 'overview',
 'topical',
 'guide',
 'computer',
 'vision',
 'computer',
 'vision',
 '–',
 'interdisciplinary',
 'field',
 'deals',
 'computers',
 'made',
 'gain',
 'high-level',
 'understanding',
 'digital',
 'images',
 'videos',
 'perspective',
 'engineering',
 'seeks',
 'automate',
 'tasks',
 'human',
 'visual',
 'system',
 'computer',
 'vision',
 'tasks',
 'include',
 'methods',
 'acquiring',
 'digital',
 'images',
 'image',
 'sensors',
 'image',
 'processing',
 'image',
 'analysis',
 'reach',
 'understanding',
 'digital',
 'images',
 'general',
 'deals',
 'extraction',
 'high-dimensional',
 'data',
 'real',
 'world',
 'order',
 'produce',
 'numerical',
 'symbolic',
 'information',
 'computer',
 'interpret',
 'image',
 'data',
 'take',
 'many',
 'forms',
 'video',
 'sequences',
 'views',
 'multiple',
 'cameras',
 'multi-dimensional',
 'data',
 'medical',
 'scanner',
 'technological',
 'discipline',
 'computer',
 'vision',
 'seeks',
 'apply',
 '

In [46]:
#df.drop('article',axis=1)
df['article']=articles

C:\Users\alecr\.conda\envs\pytorch_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


next: sort columns by count frequency

next make a dataframe with the most frequent words

In [ ]:
Something like this https://www.aclweb.org/anthology/W15-1526.pdf would be good to try - supposedly outperforms LDA, but

In [47]:
df['length'] = articles.apply(lambda x: len(x))

C:\Users\alecr\.conda\envs\pytorch_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
df['length'].describe()

count     7316.000000
mean      1324.216785
std       1505.295536
min          4.000000
25%        346.000000
50%        808.000000
75%       1711.000000
max      17054.000000
Name: length, dtype: float64

With this, discarding articles with less than a threshold of non-stopwords will help train an algorithm

In [56]:
df['original_article'][0]

"The following outline is provided as an overview of and topical guide to computer vision: Computer vision – interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do. Computer vision tasks include methods for acquiring digital images(through image sensors), image processing, and image analysis, to reach an understanding of digital images. In general, it deals with the extraction of high-dimensional data from the real world in order to produce numerical or symbolic information that the computer can interpret. The image data can take many forms, such as video sequences, views from multiple cameras, or multi-dimensional data from a medical scanner. As a technological discipline, computer vision seeks to apply its theories and models for the construction of computer vision systems. As a scientific discipline, computer v

In [57]:
df.article[0]

['following',
 'outline',
 'provided',
 'overview',
 'topical',
 'guide',
 'computer',
 'vision',
 'computer',
 'vision',
 '–',
 'interdisciplinary',
 'field',
 'deals',
 'computers',
 'made',
 'gain',
 'high-level',
 'understanding',
 'digital',
 'images',
 'videos',
 'perspective',
 'engineering',
 'seeks',
 'automate',
 'tasks',
 'human',
 'visual',
 'system',
 'computer',
 'vision',
 'tasks',
 'include',
 'methods',
 'acquiring',
 'digital',
 'images',
 'image',
 'sensors',
 'image',
 'processing',
 'image',
 'analysis',
 'reach',
 'understanding',
 'digital',
 'images',
 'general',
 'deals',
 'extraction',
 'high-dimensional',
 'data',
 'real',
 'world',
 'order',
 'produce',
 'numerical',
 'symbolic',
 'information',
 'computer',
 'interpret',
 'image',
 'data',
 'take',
 'many',
 'forms',
 'video',
 'sequences',
 'views',
 'multiple',
 'cameras',
 'multi-dimensional',
 'data',
 'medical',
 'scanner',
 'technological',
 'discipline',
 'computer',
 'vision',
 'seeks',
 'apply',
 '